In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
#from sklearn import preprocessing
from sklearn.metrics import f1_score
from IPython.display import Image
import graphviz
import pydot
#import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import os
from dotenv import load_dotenv

load_dotenv() # load environment variables

Python-dotenv could not parse statement starting at line 8


True

In [13]:
df = pd.read_csv("/home/trucbui/PBLGit/random_forest/binary_classification_train.csv", sep="\t")
df = df.sort_values(['Label','Header'], ascending=(False,True))
df=df.reset_index(drop=True)
print(df)

       Label      Header  Sequence length        Mass Most frequent aa  \
0          1  A0A024RBG1              181   20403.285                E   
1          1  A0A024SMV2              391   42741.476                A   
2          1  A0A060S684              585   63631.507                A   
3          1  A0A075TXZ3              561   61430.813                L   
4          1  A0A077K8G3              407   45202.745                L   
...      ...         ...              ...         ...              ...   
48701      0      X2JAU8             1075  117164.578                E   
48702      0      X5HYT8              425   46670.153                A   
48703      0      X5LX76             1029  118226.052                S   
48704      0      X6R8D5              127   14504.179                S   
48705      0      X6R8R1              474   52161.620                L   

       Last aa is H  Last aa is Y  
0                 0             0  
1                 0             0  
2  

In [3]:
'''
#Convert non numeric data into numeric data    
#TODO: find another way to convert. The numeric data should not be ranked
non_numeric_set=[df['Most frequent aa']]
le = preprocessing.LabelEncoder()
after_processing=[]
for s in non_numeric_set:
  after_processing.append(le.fit_transform(s))
df['Most frequent aa']=after_processing[0]

print(df)
'''


"\n#Convert non numeric data into numeric data    \n#TODO: find another way to convert. The numeric data should not be ranked\nnon_numeric_set=[df['Most frequent aa']]\nle = preprocessing.LabelEncoder()\nafter_processing=[]\nfor s in non_numeric_set:\n  after_processing.append(le.fit_transform(s))\ndf['Most frequent aa']=after_processing[0]\n\nprint(df)\n"

<font size="5">Working with Embeddings</font>

In [12]:
def read_esm2(path_to_esm2:str) -> pd.DataFrame:
    
    with h5py.File(path_to_esm2) as hdf_handle:
        embeddings_max = []
        embeddings_min = []
        embeddings_median=[]
        embeddings_deviation=[]
    
        for header, emb in hdf_handle.items():
            emb = np.array(list(emb))
            embeddings_max.append(np.amax(emb))
            embeddings_min.append(np.min(emb))
            embeddings_median.append(np.median(emb))
            embeddings_deviation.append(np.std(emb))


        return pd.DataFrame(data={"Emb max": embeddings_max, "Emb min": embeddings_min, "Emb median": embeddings_median, "Emb std":embeddings_deviation})
    

embeddings_esm2_enzymes_30 = read_esm2(os.getenv("ESM2_ENZYMES_SPLIT_30"))
embeddings_esm2_non_enzymes = read_esm2(os.getenv("ESM2_NON_ENZYMES"))


In [16]:
emb_all=pd.concat([embeddings_esm2_enzymes_30,embeddings_esm2_non_enzymes],axis=0) #concat vertically
emb_all=emb_all.reset_index(drop=True)
print(emb_all)

         Emb max   Emb min  Emb median   Emb std
0      12.089359 -0.613488   -0.003883  0.246559
1      10.992490 -0.462509   -0.002613  0.224955
2      11.732064 -0.775824   -0.002250  0.238975
3      11.502498 -0.975507   -0.001353  0.235385
4      12.043853 -0.965225   -0.004369  0.246203
...          ...       ...         ...       ...
48701   9.954324 -1.101609   -0.003210  0.218149
48702  13.143876 -0.896627   -0.003682  0.267518
48703  12.641706 -0.686069   -0.003806  0.255798
48704  13.531825 -1.542869   -0.004650  0.287314
48705   9.344967 -0.876774   -0.001748  0.203606

[48706 rows x 4 columns]


In [17]:
df=pd.concat([df,emb_all],axis=1)
print(df)

       Label      Header  Sequence length        Mass Most frequent aa  \
0          1  A0A024RBG1              181   20403.285                E   
1          1  A0A024SMV2              391   42741.476                A   
2          1  A0A060S684              585   63631.507                A   
3          1  A0A075TXZ3              561   61430.813                L   
4          1  A0A077K8G3              407   45202.745                L   
...      ...         ...              ...         ...              ...   
48701      0      X2JAU8             1075  117164.578                E   
48702      0      X5HYT8              425   46670.153                A   
48703      0      X5LX76             1029  118226.052                S   
48704      0      X6R8D5              127   14504.179                S   
48705      0      X6R8R1              474   52161.620                L   

       Last aa is H  Last aa is Y    Emb max   Emb min  Emb median   Emb std  
0                 0             

<font size="5">Train the model</font>

In [18]:
#Define the features and output:
y = np.array(df['Label'])
X = df.drop(['Label','Header','Sequence length','Most frequent aa'],axis=1)
#Split data into train an test, with test size of 20%:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
#Build the model:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred=rf.predict(X)

<font size="5">Evaluate the model</font>

In [19]:
print("accuracy on training set: %f" % rf.score(X_train, y_train))
print("accuracy on test set: %f" % rf.score(X_test, y_test))
f1 = f1_score(y, y_pred, zero_division="warn")
print('F1 score:', f1)

accuracy on training set: 0.999923
accuracy on test set: 0.823342
F1 score: 0.9020120495623508


<font size="5">Make a tree</font>

In [21]:
#Using only 5 nodes:
rf_5 = RandomForestClassifier(max_depth=5)
rf_5.fit(X_train, y_train)
# Pull out one tree from the forest:
tree = rf_5.estimators_[5]
# Export the image to a dot file:
export_graphviz(tree, out_file = 'tree.dot', feature_names = X.columns, rounded = True, precision = 1)
# Use dot file to create a graph:
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file:
graph.write_png('tree_rf.png')